In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

load_data = pd.read_csv("load.csv")
price_data = pd.read_csv("price.csv")

load_data.time = pd.to_datetime(load_data.time)
price_data.time = pd.to_datetime(price_data.time)

load_data['date'] = load_data['time'].dt.date
load_data['hour'] = load_data['time'].dt.strftime("%H")
load_data['minute'] = load_data['time'].dt.strftime("%M")
load_data['seconds'] = load_data['time'].dt.strftime("%S")
price_data['date'] = price_data['time'].dt.date
price_data['hour'] = price_data['time'].dt.strftime("%H")
price_data['minute'] = price_data['time'].dt.strftime("%M")
price_data['seconds'] = price_data['time'].dt.strftime("%S")

new_prices = price_data.loc[ (price_data.seconds == '00') & (price_data.minute == '00')]
new_prices.reset_index(drop=True)

new_load = load_data.loc[ (load_data.seconds == '00') & (load_data.minute == '00')]
new_load.reset_index(drop=True)

combined = new_load
combined = combined.merge(new_prices, left_on='time', right_on='time')
combined = combined.drop_duplicates(subset=['time'], keep='first')
combined.reset_index(drop=True)

combined = combined.rename(columns={"value_x" : "load", "date_x" : "date", "hour_x" : "hour", "minute_x" : "minute", "seconds_x" : "second", "value_y" : "price"})
combined = combined.drop(columns={"label_x", "type_x", "label_y", "type_y", "date_y", "hour_y", "minute_y", "seconds_y"})
combined = combined.drop(columns={"minute", "second"})
combined = combined[['time', 'date', 'hour', 'load', 'price']]
combined.hour = combined.hour.astype('int')
combined.hour.replace(0,24, inplace=True)

dr = pd.date_range(start='2020-11-01', end='2023-11-01')
cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

combined['holiday'] = combined['date'].astype('datetime64').isin(holidays).astype('int')

combined['day_of_week'] = combined.time.dt.dayofweek + 1

In [2]:
combined = combined.reset_index(drop=True)

In [3]:
combined2 = combined.iloc[24*7:,:]

In [4]:
combined2['load(d-1,h)'] = -1.0
combined2['load(d-7,h)'] = -1.0
combined2['price(d-1,h)'] = -1.0
combined2['price(d-7,h)'] = -1.0
combined2['avg_load(d-1)'] = -1.0
combined2['avg_price(d-1)'] = -1.0

/var/folders/vn/24pgf0213xzgjbvpt_5fqd640000gn/T/ipykernel_2759/1562718623.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined2['load(d-1,h)'] = -1.0
/var/folders/vn/24pgf0213xzgjbvpt_5fqd640000gn/T/ipykernel_2759/1562718623.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined2['load(d-7,h)'] = -1.0
/var/folders/vn/24pgf0213xzgjbvpt_5fqd640000gn/T/ipykernel_2759/1562718623.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [5]:
for index, row in combined2.iterrows():
    combined2.at[index,'load(d-1,h)'] = combined.iloc[index - 24, 3]
    combined2.at[index,'load(d-7,h)'] = combined.iloc[index - 24*7, 3]
    combined2.at[index,'price(d-1,h)'] = combined.iloc[index - 24, 4]
    combined2.at[index,'price(d-7,h)'] = combined.iloc[index - 24*7, 4]

In [6]:
combined2 = combined2[['time', 'date', 'day_of_week', 'hour', 'holiday', 'load(d-1,h)', 'load(d-7,h)', 'avg_load(d-1)', 'price(d-1,h)', 'price(d-7,h)', 'avg_price(d-1)', 'load', 'price']]

In [7]:
avg_load = combined.iloc[167-24 : 167, 3].mean()
avg_price = combined.iloc[167-24 : 167, 4].mean()

for index, row in combined2.iterrows():
    if (index >= 191) and ((index + 1) % 24 == 0):
        avg_load = combined.iloc[index-24 : index, 3].mean()
        avg_price = combined.iloc[index-24 : index, 4].mean()
    
    combined2.at[index, 'avg_load(d-1)'] = avg_load
    combined2.at[index, 'avg_price(d-1)'] = avg_price


In [8]:
combined2.to_csv("nyc_load_price.csv")